Составьте прогноз метрик рассматриваемого игрового проекта с использованием нуклеарной модели
для пользователей СНГ и Северной Америки на платформах Android (CPI = 1,5$) и iOS (CPI =
3$). В качестве фактических
данных используйте период январь-февраль 2021 г., а прогноз составьте на март-апрель 2021 г.
Опишите полученные результаты, сравните их с реальными данными за март-апрель 2021 г

In [2]:
#новые пользователи+затраты
pd.read_sql(f'''
SELECT *
FROM(
    select
        date_trunc('month', event_date)::date monthes,
        case WHEN country in ('RU', 'AZ', 'AM', 'BY','KZ', 'KG','MD', 'TJ', 'UZ', 'UA', 'TM') then 'CIS'
             WHEN country in ('CA', 'US') then 'NA'
             else 'Other' end as region,
        platform,
        count(distinct case when event_name = 'FirstLaunchApp' and user_type =
        'organic' then user_id end) new_organic,
        count(distinct case when event_name = 'FirstLaunchApp' and user_type =
        'non_organic' then user_id end) new_nonorganic,
        count(distinct case when event_name = 'FirstLaunchApp' and user_type =
        'non_organic' and platform='android' then user_id end)*1.5 marketing_budget_usd_android,
        count(distinct case when event_name = 'FirstLaunchApp' and user_type =
        'non_organic' and platform='ios' then user_id end)*3 marketing_budget_usd_ios,
        count(distinct case when event_name = 'FirstLaunchApp' then user_id end)
        new_users
    from data_viz_1068.project_dataset
    group by 1,2,3
    order by 1) as t
WHERE region!='Other' and monthes between '2021-01-01' and '2021-04-01'
''', con=engine)

,monthes,region,platform,new_organic,new_nonorganic,marketing_budget_usd_android,marketing_budget_usd_ios,new_users
0,2021-01-01,CIS,android,1582,46,69.0,0,1627
1,2021-01-01,CIS,ios,880,9,0.0,27,888
2,2021-01-01,NA,android,271,10,15.0,0,278
3,2021-01-01,NA,ios,911,5,0.0,15,916
4,2021-02-01,CIS,android,1227,28,42.0,0,1253
5,2021-02-01,CIS,ios,639,7,0.0,21,646
6,2021-02-01,NA,android,244,3,4.5,0,247
7,2021-02-01,NA,ios,641,3,0.0,9,644
8,2021-03-01,CIS,android,3156,32,48.0,0,3186
9,2021-03-01,CIS,ios,1085,7,0.0,21,1092


In [3]:
#ядро, непостоянные,вернувшиеся
pd.read_sql(f'''
with logins as
    (
    SELECT *
    FROM(
        select distinct date_trunc('month', event_date)::date monthes,
            case WHEN country in ('RU', 'AZ', 'AM', 'BY','KZ', 'KG','MD', 'TJ', 'UZ', 'UA', 'TM') then 'CIS'
                 WHEN country in ('CA', 'US') then 'NA'
                 else 'Other' end as region,
            platform, user_id
        from
        data_viz_1068.project_dataset) as r
    WHERE region!='Other' 
),
users_types as
    (
    SELECT monthes, user_id,
    region, platform,
    case when DATE_PART('month',AGE(monthes,lag1))=1
             and DATE_PART('month',AGE(monthes,lag2))=2
             and DATE_PART('month',AGE(monthes,lag3))=3 
    then 'core'
    when DATE_PART('month',AGE(monthes,lag1)) in (1,2,3) then 'fickle'
    when DATE_PART('month',AGE(monthes,lag1))>3 then 'return' else null end as user_group
    FROM
        (select monthes, user_id,region,
            platform,
        lag(monthes,1) over(partition by user_id,region, platform order by monthes) as lag1,
        lag(monthes,2) over(partition by user_id,region, platform order by monthes) as lag2,
        lag(monthes,3) over(partition by user_id,region, platform order by monthes) as lag3
        from logins) as t
    )

SELECT monthes, region,platform, 
count(case when user_group = 'core' then user_id end) core_users,
count(case when user_group = 'fickle' then user_id end) fickle_users,
count(case when user_group = 'return' then user_id end) return_users
FROM users_types
WHERE monthes between '2021-01-01' and '2021-04-01'
GROUP BY 1,2,3
ORDER BY 1
''', con=engine)

,monthes,region,platform,core_users,fickle_users,return_users
0,2021-01-01,CIS,android,1,2218,0
1,2021-01-01,CIS,ios,217,489,0
2,2021-01-01,NA,android,0,697,0
3,2021-01-01,NA,ios,316,756,0
4,2021-02-01,CIS,android,650,928,0
5,2021-02-01,CIS,ios,242,321,9
6,2021-02-01,NA,android,251,246,0
7,2021-02-01,NA,ios,402,473,10
8,2021-03-01,CIS,android,682,717,90
9,2021-03-01,CIS,ios,198,278,40


In [4]:
#коэфициенты
pd.read_sql(f'''
with clean_data as
    (
    SELECT *
    FROM(
        select distinct date_trunc('month', event_date)::date monthes,
            case WHEN country in ('RU', 'AZ', 'AM', 'BY','KZ', 'KG','MD', 'TJ', 'UZ', 'UA', 'TM') then 'CIS'
                 WHEN country in ('CA', 'US') then 'NA'
                 else 'Other' end as region,
            platform, user_id,
            event_name,
            revenue_usd
        from
        data_viz_1068.project_dataset) as r
    WHERE region!='Other' and event_name in ('FirstLaunchApp', 'LaunchApp', 'af_purchase')
),
users_types as
    (
    SELECT monthes, user_id,
    region, platform,
    case when DATE_PART('month',AGE(monthes,lag1))=1
             and DATE_PART('month',AGE(monthes,lag2))=2
             and DATE_PART('month',AGE(monthes,lag3))=3 
    then 'core'
    when DATE_PART('month',AGE(monthes,lag1)) in (1,2,3) then 'fickle'
    when DATE_PART('month',AGE(monthes,lag1))>3 then 'return' else null end as user_group
    FROM
        (select monthes, user_id,region,
            platform,
        lag(monthes,1) over(partition by user_id,region, platform order by monthes) as lag1,
        lag(monthes,2) over(partition by user_id,region, platform order by monthes) as lag2,
        lag(monthes,3) over(partition by user_id,region, platform order by monthes) as lag3
        from clean_data) as t
),

coefficient_users as
(
    select
        *,
        case
        when (user_group = 'core' and (lag(user_group, 1) over(partition by user_id,region, platform order by monthes desc) <> 'core'))
             or (user_group = 'core' and max(monthes) over(partition by user_id,region, platform) = monthes 
             and monthes <> (select max(monthes) from users_types))
        then 'core decay'
        
        when user_group = 'fickle' and lag(user_group, 1) over(partition by user_id,region, platform order by monthes desc) = 'core'
        then 'transform to core'
        
        when user_group = 'fickle' and max(monthes) over(partition by user_id,region, platform) = monthes 
             and monthes <> (select max(monthes) from users_types)
        then 'fickle decay'
        
        when user_group = 'return' and lag(user_group, 1) over(partition by user_id,region, platform order by monthes desc) = 'fickle'
        then 'transform return to fickle'
        
        when user_group = 'new' and lag(user_group, 1) over(partition  by user_id,region, platform order by monthes desc) = 'fickle'
        then 'transform new to fickle'
        
        end as coefficients
    from users_types
)
select monthes,region, platform,
    case
    when count(case when user_group = 'core' then user_id end) > 0
    then 1 - count(case when coefficients = 'core decay' then user_id
    end)::float/count(case when user_group = 'core' then user_id end)::float
    else 0
    end as core_decay,
    
    case
    when count(case when user_group = 'fickle' then user_id end) > 0
    then count(case when coefficients = 'transform to core' then user_id
    end)::float/count(case when user_group = 'fickle' then user_id end)::float
    else 0
    end as transform_to_core,
    
    case
    when count(case when user_group = 'fickle' then user_id end) > 0
    then 1 - count(case when coefficients = 'fickle decay' then user_id
    end)::float/count(case when user_group = 'fickle' then user_id end)::float
    else 0
    end as fickle_decay,
    
    case
    when count(case when user_group = 'return' then user_id end) > 0
    then count(case when coefficients = 'transform return to fickle'
    then user_id end)::float/count(case when user_group = 'return' then user_id
    end)::float
    else 0
    end as transform_return_to_fickle,
    
    case
    when count(case when user_group = 'new' then user_id end) > 0
    then count(case when coefficients = 'transform new to fickle' then
    user_id end)::float/count(case when user_group = 'new' then user_id end)::float
    else 0
    end as transform_new_to_fickle
    
from coefficient_users
WHERE monthes between '2021-01-01' and '2021-04-01' and region!='Other'
GROUP BY 1,2,3
ORDER BY 1


''', con=engine)

,monthes,region,platform,core_decay,transform_to_core,fickle_decay,transform_return_to_fickle,transform_new_to_fickle
0,2021-01-01,CIS,android,0.000000,0.196034,0.557458,0.000000,0.0
1,2021-01-01,CIS,ios,0.649351,0.157609,0.490942,0.000000,0.0
2,2021-01-01,NA,android,0.000000,0.198276,0.589080,0.000000,0.0
3,2021-01-01,NA,ios,0.620853,0.159117,0.557491,0.000000,0.0
4,2021-02-01,CIS,android,0.722222,0.148050,0.566489,0.000000,0.0
5,2021-02-01,CIS,ios,0.648649,0.092593,0.449735,0.000000,0.0
6,2021-02-01,NA,android,0.600000,0.123249,0.588235,0.000000,0.0
7,2021-02-01,NA,ios,0.690566,0.114754,0.511475,0.400000,0.0
8,2021-03-01,CIS,android,0.676829,0.050773,0.415011,0.166667,0.0
9,2021-03-01,CIS,ios,0.651613,0.105919,0.411215,0.150000,0.0


In [5]:
#денюшки для новых пользователей
pd.read_sql(f'''
with clean_data as
    (
    SELECT *
    FROM(
        select distinct date_trunc('month', event_date)::date monthes,
            case WHEN country in ('RU', 'AZ', 'AM', 'BY','KZ', 'KG','MD', 'TJ', 'UZ', 'UA', 'TM') then 'CIS'
                 WHEN country in ('CA', 'US') then 'NA'
                 else 'Other' end as region,
            platform, user_id,
            event_name,
            revenue_usd
        from data_viz_1068.project_dataset) as r
    WHERE region!='Other' and event_name in ('FirstLaunchApp', 'af_purchase')
),
new_users as
(
    select distinct monthes, user_id, region, platform
    from clean_data
    where event_name = 'FirstLaunchApp'
),
payments as
(
    select monthes, user_id, sum(revenue_usd) revenue_usd, region, platform
    from clean_data
    group by 1,2,4,5
)

SELECT *
FROM
(select nu.monthes,nu.platform, nu.region,
    sum(revenue_usd) revenue_usd_monthly,
    count(case when revenue_usd>0 then nu.user_id end)::float/count(nu.user_id)::float new_users_ppu,
    case when count(case when revenue_usd>0 then nu.user_id end)>0
    then sum(revenue_usd)/count(case when revenue_usd>0 then nu.user_id
    end) else 0
    end as new_users_arppu
from new_users nu
left join payments p
ON
nu.monthes = p.monthes and nu.user_id = p.user_id and nu.platform = p.platform and nu.region = p.region
group by 1,2,3
order by 1) as w
WHERE monthes between '2021-01-01' and '2021-04-01'


''', con=engine)

,monthes,platform,region,revenue_usd_monthly,new_users_ppu,new_users_arppu
0,2021-01-01,android,CIS,1123.378078,0.041180,16.766837
1,2021-01-01,android,NA,1565.879578,0.111511,50.512244
2,2021-01-01,ios,CIS,900.848350,0.028153,36.033934
3,2021-01-01,ios,NA,2848.410708,0.078603,39.561260
4,2021-02-01,android,CIS,378.745258,0.030327,9.966980
5,2021-02-01,android,NA,1044.401776,0.101215,41.776071
6,2021-02-01,ios,CIS,358.200767,0.024768,22.387548
7,2021-02-01,ios,NA,2039.858353,0.077640,40.797167
8,2021-03-01,android,CIS,964.407437,0.031701,9.548588
9,2021-03-01,android,NA,1455.334452,0.138728,60.638936


In [18]:
#денюшки для остальных
pd.read_sql(f'''
with clean_data as
    (
    SELECT *
    FROM(
        select distinct date_trunc('month', event_date)::date monthes,
            case WHEN country in ('RU', 'AZ', 'AM', 'BY','KZ', 'KG','MD', 'TJ', 'UZ', 'UA', 'TM') then 'CIS'
                 WHEN country in ('CA', 'US') then 'NA'
                 else 'Other' end as region,
            platform, user_id,
            event_name,
            revenue_usd
        from data_viz_1068.project_dataset) as r
    WHERE region!='Other' and event_name in ('LaunchApp', 'af_purchase')
    and (monthes, user_id) not in
                    (SELECT *
                    FROM(
                        select distinct date_trunc('month', event_date)::date monthes, user_id
                        from data_viz_1068.project_dataset) as r
                        WHERE event_name in ('FirstLaunchApp'))    
),
users_types as
    (
    SELECT monthes, user_id,
    region, platform,
    case when DATE_PART('month',AGE(monthes,lag1))=1
             and DATE_PART('month',AGE(monthes,lag2))=2
             and DATE_PART('month',AGE(monthes,lag3))=3 
    then 'core'
    when DATE_PART('month',AGE(monthes,lag1)) in (1,2,3) then 'fickle'
    when DATE_PART('month',AGE(monthes,lag1))>3 then 'return' else null end as user_group,
    revenue_usd as monthly_amount_usd
    FROM
        (select monthes, user_id,region,
            platform,revenue_usd,
        lag(monthes,1) over(partition by user_id,region, platform order by monthes) as lag1,
        lag(monthes,2) over(partition by user_id,region, platform order by monthes) as lag2,
        lag(monthes,3) over(partition by user_id,region, platform order by monthes) as lag3
        from clean_data) as t
)

SELECT  monthes, platform, region,
        sum(case when user_group = 'core' then monthly_amount_usd end) as core_revenue,
        sum(case when user_group = 'fickle' then monthly_amount_usd end) as fickle_revenue,
        sum(case when user_group = 'return' then monthly_amount_usd end) as return_revenue,
        
        case WHEN count(case when user_group = 'core' then user_id end)>0
        then count(case when user_group = 'core' and
                monthly_amount_usd > 0 then user_id end)::float/count(case when user_group ='core' then user_id end)::float
             ELSE 0
        end as core_users_ppu,
        
        case WHEN count(case when user_group = 'fickle' then user_id end)>0
             then count(case when user_group = 'fickle' and
             monthly_amount_usd > 0 then user_id end)::float/count(case when user_group ='fickle' then user_id end)::float
             ELSE 0
        end as fickle_users_ppu,
        
        case WHEN count(case when user_group = 'return' then user_id end)>0
             then count(case when user_group = 'return' and
             monthly_amount_usd > 0 then user_id end)::float/count(case when user_group ='return' then user_id end)::float
             ELSE 0
        end as return_users_ppu,
        
        case WHEN count(case when user_group = 'core' and monthly_amount_usd > 0
             then user_id end)>0
             then sum(case when user_group = 'core' then monthly_amount_usd
             end)::float/count(case when user_group = 'core' and monthly_amount_usd > 0 then
             user_id end)::float
             ELSE 0
        end as core_users_arppu,
        
        case WHEN count(case when user_group = 'fickle' and monthly_amount_usd > 0
             then user_id end)>0
             then sum(case when user_group = 'fickle' then
             monthly_amount_usd end)::float/count(case when user_group = 'fickle' and
             monthly_amount_usd > 0 then user_id end)::float
             ELSE 0
        end as fickle_users_arppu,
        
        case WHEN count(case when user_group = 'return' and monthly_amount_usd > 0
             then user_id end)>0
             then sum(case when user_group = 'return' then
             monthly_amount_usd end)::float/count(case when user_group = 'return' and
             monthly_amount_usd > 0 then user_id end)::float
             ELSE 0
        end as return_users_arppu
from
users_types
WHERE monthes between '2021-01-01' and '2021-04-01' 
group by 1,2,3
order by 1

''', con=engine)

,monthes,platform,region,core_revenue,fickle_revenue,return_revenue,core_users_ppu,fickle_users_ppu,return_users_ppu,core_users_arppu,fickle_users_arppu,return_users_arppu
0,2021-01-01,android,CIS,NaN,235.430549,NaN,0.000000,0.024586,0.000000,0.000000,4.527511,0.000000
1,2021-01-01,android,NA,NaN,582.557453,NaN,0.000000,0.099407,0.000000,0.000000,8.694887,0.000000
2,2021-01-01,ios,CIS,21.117616,174.952757,NaN,0.017442,0.047431,0.000000,7.039205,7.289698,0.000000
3,2021-01-01,ios,NA,26.000000,826.060403,NaN,0.018182,0.096894,0.000000,6.500000,10.590518,0.000000
4,2021-02-01,android,CIS,11.561875,188.462007,NaN,0.005464,0.036649,0.000000,3.853958,5.384629,0.000000
5,2021-02-01,android,NA,1.990000,463.072123,NaN,0.006211,0.113208,0.000000,1.990000,12.863115,0.000000
6,2021-02-01,ios,CIS,1.327407,50.144938,0.000000,0.004950,0.028571,0.000000,1.327407,5.571660,0.000000
7,2021-02-01,ios,NA,18.000000,557.153898,0.000000,0.010909,0.087500,0.000000,6.000000,11.370488,0.000000
8,2021-03-01,android,CIS,10.892165,82.079468,0.000000,0.003425,0.029289,0.000000,5.446083,3.908546,0.000000
9,2021-03-01,android,NA,26.950000,341.302801,0.000000,0.034247,0.120690,0.000000,5.390000,12.189386,0.000000


https://docs.google.com/spreadsheets/d/1pg0OppcfSo5ZSUDiCQ9Ec1vKoL9qpWUytYB-M56qFPo/edit?usp=sharing